# ELYZA-tasks-100 による性能評価

In [ ]:
!pip install accelerate datasets jsonlines sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.9 MB/s eta 0:00:00


## ELYZA-tasks-100 データセットのダウンロード

モデルのダウンロードには 🤗datasets ライブラリを用いる。

データは `/content/dataset.jsonl` に保存する。

In [ ]:
import json
from pathlib import Path

from datasets import load_dataset

# Load the dataset
ds = load_dataset("elyza/ELYZA-tasks-100")

# Function to convert dataset to JSONL format and print
def dataset_to_jsonl(dataset, filename):
    with open(filename, 'w', encoding='utf-8') as file:
        for entry in dataset:
            # Construct JSON object
            json_obj = {
                "input_text": entry['input'],
                "output_text": entry['output'],
                "eval_aspect": entry['eval_aspect']
            }

            # Write JSON object to file in JSONL format
            json_str = json.dumps(json_obj, ensure_ascii=False)
            file.write(json_str + '\n')

# Convert and write the dataset to a file in JSONL format
path_jsonl = Path("/content/dataset.jsonl")

if (path_jsonl.parent is not None) and (not path_jsonl.parent.exists()):
    path_jsonl.parent.mkdir(parents=True, exist_ok=True)

dataset_to_jsonl(ds["test"], path_jsonl)


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating test split: 0 examples [00:00, ? examples/s]

## 評価

ELYZA-tasks-100 データセットを用いて、指定したモデルの評価を実行する。

### プロンプトについて

モデルに回答を生成させるときのプロンプトは、各モデルのモデルカード (🤗HuggingFace の README ページ) に記載されているプロンプトを使用する。

> 📒 **ノート**
>
> この方法は、質問を直接入力する場合や、全モデルで共通のプロンプトを用いる場合に比べて公平性が劣る。
> しかし、実利用時は各モデルが最もよい性能を発揮するように、各モデルのプロンプトをチューニングすることが想定される。
> モデルカード記載のプロンプトをチューニング済みのプロンプトと見なすことで、実利用時の性能を比較することを狙いとする。

### ハイパーパラメータについて

ハイパーパラメータは 🤗HuggingFace の Code Snippet のとおりとする（公開者が指定したハイパーパラメータを、チューン済みとみなす）。
ただし、長文の回答を要求する設問もあるため、`max_new_tokens` は `1024` に統一する。

In [ ]:
import json
import jsonlines
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Setup model and tokenizer
model_name = "elyza/ELYZA-japanese-Llama-2-7b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
    device_map="auto",
    # trust_remote_code=True, # ⚠️ Microsoft/Phi-1.5 のみ必要
)

def format_prompt(input_text):
    # ✋プロンプトは、モデルに合わせて調整（HuggingFace のモデルカード参照）
    prompt_template = "<s>[INST] <<SYS>>\nあなたは誠実で優秀な日本人のアシスタントです。\n<</SYS>>\n\n{input} [/INST] "
    return prompt_template.format(input=input_text)

def generate_text(input_text):
    formatted_prompt = format_prompt(input_text)
    input_ids = tokenizer.encode(
        formatted_prompt,
        add_special_tokens=False,
        return_tensors="pt"
    )

    # Set seed for reproducibility
    seed = 42
    torch.manual_seed(seed)

    # ✋Hparamsは、モデルに合わせて調整（HuggingFace のモデルカード参照）
    tokens = model.generate(
        input_ids.to(device=model.device),
        max_new_tokens=1024,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id
    )

    # Remove the input tokens from the generated tokens before decoding
    output_tokens = tokens[0][len(input_ids[0]):]
    return tokenizer.decode(output_tokens, skip_special_tokens=True)

def process_dataset(input_file, output_file):
    # Read dataset and generate predictions
    with jsonlines.open(input_file) as reader, jsonlines.open(output_file, mode='w') as writer:
        for obj in reader:
            prompt = obj['input_text']
            generated_text = generate_text(prompt)
            print(generated_text)
            writer.write({"pred": generated_text})

# Process the dataset
input_dataset = '/content/dataset.jsonl'
output_predictions = '/content/preds.jsonl'
process_dataset(input_dataset, output_predictions)

tokenizer_config.json:   0%|          | 0.00/725 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/641 [00:00<?, ?B/s]

...

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/154 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


仕事の熱意を取り戻すためのアイデアをご紹介します。

1. 仕事のやりがいを見つける: 仕事のやりがいを見つけることで、仕事に対する意欲が高まり、熱意が湧き上がってきます。

2. 仕事の進め方を見直す: 自分の仕事の進め方を見直し、効率の良い方法を見つけることで、仕事への取り組みやすさが変わり、熱意が高まる可能性があります。

3. 仕事のやる気を出す音楽を聞く: 仕事のやる気を出すための音楽を聞くことで、気分を高揚させ、熱意を取り戻すことができるかもしれません。

4. 仕事の合間に休憩する: 仕事の合間に休憩することで、仕事に対する熱意を取り戻すだけでなく、集中力を回復させ、仕事の効率を上げることもできます。

5. 仕事のやりがいを伝える: 自分が仕事のやりがいを感じていることを伝えることで、仕事への取り組みが活発になり、熱意が高まる可能性があります。
承知しました。以下にクマが海辺に行ってアザラシと友達になり、最終的には家に帰るというプロットの短編小説を記述します。

クマは山の中でゆっくりと眠っていた。
その眠りに落ちたクマは、夢の中で海辺を歩いていた。
そこにはアザラシがいた。
クマはアザラシに話しかける。

「おはよう」とクマが言うと、アザラシは驚いたように顔を上げた。
「あ、おはよう」アザラシは少し緊張した様子だった。
クマはアザラシと友達になりたいと思う。

「私はクマと申します。」クマは自己紹介をする。
アザラシは顔をほころばせた。
「私はアザラシと申します。」アザラシも自己紹介をする。
クマはアザラシと一緒に遊びたいと思う。

「アザラシ、今日は一緒に遊ぼうよ。」クマはアザラシに誘う。
アザラシは顔をほころばせた。
「ええっ、いいの？」アザラシは嬉しそうに言う。
クマはアザラシの返事を聞いて嬉しそうに笑う。

クマとアザラシは海辺で遊ぶ。
二匹は砂浜を走り、波をかく。
二匹は海の中に潜った。
二匹は魚を捕まえた。
二匹は海辺で食事をした。
二匹は海辺で寝転がった。
二匹は海辺で星を眺めた。
二匹は海辺で笑い合った。

クマはアザラシと一緒に過ごした時間が楽しいと思う。
アザラシもクマと一緒に過ごした時間が楽しいと思う。

クマはアザラシに告白する。

「アザラシ、私は君のことが好きになりました。」クマはアザラシの顔を見つめながら言う。
アザラ